# Стадия II

Программа принимает на вход начальные условия и промежуток времени и выводит сведения об опасных сближениях, которые случились в этот промежуток времени: ситуациях, когда расстояния между объектами меньше некоторой константы умноженной на сумму размеров объектов. По умолчанию, константа равна 3. Расчёт на GPU.
Формат: Id первого объекта, Id второго объекта, время начала опасного сближения, время конца опасного сближения, минимальное расстояние между объектами, скорости и координаты объектов в момент минимального расстояния, время, в которое расстояние минимально
В отчёте: вариант случайных начальных условий и список следующих не более 1000 опасных ситуаций, которые случатся в течение недели. Для первого опасного сближения: визуализация траекторий с момента начала стимуляции до окончания опасного сближения. Измерение производительности отдельно интегратора и поиска опасных ситуаций.
